# Japan Data for Yolo 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import json
import cv2
import os

In [2]:
def extract_data_from_json(json_file):
    # JSON-Datei laden
    with open(json_file, 'r') as file:
        data = json.load(file)

    img_width = data['imageWidth']
    img_height = data['imageHeight']
    
    # Leere Liste zum Speichern aller Polygonpunkte
    all_polygons = []
    
    # Iteriere über alle Shapes in der JSON
    for shape in data['shapes']:
        # Extrahiere die Punkte für das Polygon
        points = shape['points']
        all_polygons.append(points)
    
    return all_polygons, img_height, img_width


def generate_white_polygons(json_file, save_img=False):
    # Polygone und Bilddimensionen extrahieren
    polygons, img_height, img_width = extract_data_from_json(json_file)

    # Erstelle ein schwarzes Bild mit den gegebenen Dimensionen
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)

    # Zeichne und fülle jedes Polygon weiß
    for polygon in polygons:
        # Konvertiere die Punkte des Polygons in ein numpy-Array
        points = np.array(polygon, dtype=np.int32)
        
        # Fülle den Polygonbereich weiß (255, 255, 255)
        cv2.fillPoly(image, [points], color=(255, 255, 255))
    # Optional: Zeige das Bild an
    #cv2.imshow('Image with Polygons', image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    if save_img:

        base_name = os.path.basename(json_file)
        file_name, ext = os.path.splitext(base_name)
        new_file_name = file_name + '.png'
        current_dir = os.path.dirname(json_file)
        parent_dir = os.path.dirname(current_dir)
        new_dir = os.path.join(parent_dir, "soiling_label_img")
        # Stelle sicher, dass der neue Ordner existiert
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)

        # Erstelle den vollständigen Pfad zur neuen Datei
        new_file_path = os.path.join(new_dir, new_file_name)

        output_path = new_file_path

        print(output_path)

        cv2.imwrite(output_path, image)
        


    return image, img_height, img_width


def find_white_polygons(image, img_height, img_width):
    # Bild einlesen
    height = img_height
    width = img_width

    # Bild in Graustufen umwandeln
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Schwellenwert setzen, um weiße Bereiche zu isolieren
    _, thresholded = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    # Konturen finden
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Liste für approximierte Polygone
    polygons = []

    for contour in contours:
        # Polygon aus der Kontur approximieren
        epsilon = 0.02 * cv2.arcLength(contour, True)  # Anpassung des Toleranzwerts
        polygon = cv2.approxPolyDP(contour, epsilon, True)

        # Überprüfen, ob das Polygon mindestens 3 Punkte hat
        if len(polygon) < 3:
            continue  # Polygon überspringen, wenn es weniger als 3 Punkte hat
        
        # Polygonpunkte normalisieren
        normalized_polygon = []
        for point in polygon:
            x, y = point[0]
            normalized_x = x / width
            normalized_y = y / height
            normalized_polygon.append((normalized_x, normalized_y))
        polygons.append(normalized_polygon)

    # Bild mit den Polygonen anzeigen
    for polygon in polygons:
        # Umwandeln der Normalisierten Koordinaten zurück zu Pixeln für die Anzeige
        display_polygon = np.array([(int(x * width), int(y * height)) for x, y in polygon], np.int32)
        cv2.polylines(image, [display_polygon], isClosed=True, color=(0, 255, 0), thickness=2)

    #cv2.imshow('Polygons', image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    return polygons


def safe_in_txt(polygons, json_path):
# Speichern der Polygon-Daten in einer Textdatei

    base_name = os.path.basename(json_path)
    file_name, ext = os.path.splitext(base_name)
    new_file_name = file_name + '.txt'
    current_dir = os.path.dirname(json_path)
    parent_dir = os.path.dirname(current_dir)
    new_dir = os.path.join(parent_dir, "yolo_labels")
    # Stelle sicher, dass der neue Ordner existiert
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    # Erstelle den vollständigen Pfad zur neuen Datei
    new_file_path = os.path.join(new_dir, new_file_name)

    output_path = new_file_path

    with open(output_path, 'w') as file:
        for polygon in polygons:
            # Formatieren der Zeile: Klasse 0, gefolgt von den normalisierten Koordinaten
            line = '0 ' + ' '.join(f'{x:.6f} {y:.6f}' for x, y in polygon) + '\n'
            file.write(line)


In [4]:
#img, h, w = generate_white_polygons("/home/aravind/masterthesis_klingler/data/git_japaner/labeled_ubuntu/soiling1/soiling_annotated/soiling1_10.json")
#polygons = find_white_polygons(img, h, w)
#safe_in_txt(polygons, "/home/aravind/masterthesis_klingler/data/git_japaner/labeled_ubuntu/soiling1/soiling_annotated/soiling1_10.json") 


def create_yolo_labels(main_dir):
    # Durchlaufe alle Unterverzeichnisse und -dateien
    for root, dirs, files in os.walk(main_dir):
        for file in files:
            if file.endswith(".json"):
                json_path = os.path.join(root, file)
                img, h, w = generate_white_polygons(json_path, save_img=True)
                polygons = find_white_polygons(img, h, w)
                safe_in_txt(polygons, json_path) 



#create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling1/soiling_annotated")



In [5]:
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling2/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling3/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling4/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling5/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling6/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling7/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling8/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling9/soiling_annotated")
create_yolo_labels("/home/tim/Documents/04_Projekt_ConSim/camera_contamination/data/git_japaner/labeled_ubuntu/soiling10/soiling_annotated")